# Welcome to Abide Profit Solutions. We're here to help your bottom line.

# Table of Contents
#Welcome! Thanks for signing up - Introduction to concepts
    #The process...
        #Configuration Questionnare (~1hr)
        #File Upload (~1hr)
        #Computer Running! (~1hr)
        #Analysis and Output (1hr)
        #Next steps...
    #Profit allocation
    #Making decisions with profit
    #Disclaimer before starting


# Configuration Questionnaire
See Configuration_Questions.docx

# Upload

In [32]:
import pandas as pd
import os
import plotly.graph_objects as go
transfile='uploads/transaction.csv'
plfile='uploads/pl.csv'
transactiondata=pd.read_csv(transfile,delimiter=',')
trans_df=pd.DataFrame(transactiondata)
pldata=pd.read_csv(plfile,delimiter=',')
pl_df=pd.DataFrame(pldata)

In [35]:
pl_df

,Account,Amount,FileName
0,Revenue,10000000,PL_test
1,COGS,5000000,PL_test
2,GrossMargin,5000000,PL_test
3,Distribution,2500000,PL_test
4,SellingRelated,1500000,PL_test
5,G&A,3000000,PL_test
6,Profit,-2000000,PL_test


In [3]:
#THE ENGINE ##CHOO CHOO 
accountslist=[]
def splitter2(transfile,plfile):
    transactiondata=pd.read_csv(transfile,delimiter=',')
    transdf=pd.DataFrame(transactiondata)
    pldata=pd.read_csv(plfile,delimiter=',')
    pldf=pd.DataFrame(pldata)       
    for i in pldf['Account']: 
        if i not in ("Revenue","COGS","GrossMargin","Profit"): 
            accountslist.append(i)
        else:
            continue
    transdf["GrossMargin"]=transdf['Revenue']-transdf['COGS']
    for account in accountslist:
        valuelist=[]
        totallist=[]
        totalrev=transdf['Revenue'].sum()
        for revval in transdf["Revenue"]:
            valuelist.append(round(pldf[pldf['Account']==str(account)]["Amount"].values[0]*(revval/totalrev),2))
            totallist.append(pldf[pldf['Account']==str(account)]["Amount"].values[0])
        transdf[str(account)+"AC"]=valuelist
    transdf["Profit"]=transdf['GrossMargin']-transdf.iloc[:,-len(accountslist):].sum(axis=1)
    return transdf

In [11]:
transoutput=splitter2(transfile,plfile)

In [12]:
accountslist

['Distribution',
 'SellingRelated',
 'G&A',
 'Distribution',
 'SellingRelated',
 'G&A']

# Output - Visualizing the Data

In [13]:
#Show the P&L in a waterfall
#functionalize
def plwaterfall(plfile):
    pl_df=pd.read_csv(plfile,delimiter=',')
    measurelist=[]
    amountlist=[]
    allaccountslist=list(pl_df['Account'])
    allaccountslist
    for i in allaccountslist: 
        if i in ("GrossMargin","Profit"):
            measurelist.append('total')
            amountlist.append(pl_df[pl_df['Account']==i]['Amount'].values[0])
        elif i in ("Revenue"):
            measurelist.append('relative')
            amountlist.append(pl_df[pl_df['Account']==i]['Amount'].values[0])
        else: 
            measurelist.append('relative')
            amountlist.append(-pl_df[pl_df['Account']==i]['Amount'].values[0])
    fig = go.Figure(go.Waterfall(
        name = "20", orientation = "v",
        measure = measurelist,
        x=allaccountslist,
        #textposition = "outside",
        #text = ["+60", "+80", "", "-40", "-20", "Total"],
        y = amountlist,
        connector = {"line":{"color":"rgb(63, 63, 63)"}},
    ))

    fig.update_layout(
            title = "Profit and loss statement 2019",
            showlegend = True
    )

    return fig

In [14]:
plwaterfall(plfile)

In [15]:
#customerprofit
#create dataframe
customerprofit=pd.DataFrame(transoutput.groupby('CustomerName')['Profit'].sum().reset_index().sort_values('Profit',ascending=False))
customerprofit

#ceate visualization

fig = go.Figure(data=[
    go.Bar(name='Customer Profitability', x=customerprofit['CustomerName'], y=customerprofit['Profit']),
])
# Change the bar mode
fig.update_layout(
    title='Customer Profitability',
    xaxis=dict(
        title='Customer Profitability',
        tickfont_size=14
    ),
    yaxis=dict(
    title='USD Net Profit'
    ),
    barmode='group'

)

fig.show()




In [16]:
#productprofit
#create dataframe
productprofit=pd.DataFrame(transoutput.groupby('PartName')['Profit'].sum().reset_index().sort_values('Profit',ascending=False))
customerprofit

#ceate visualization

fig = go.Figure(data=[
    go.Bar(name='Product Profitability', x=productprofit['PartName'], y=productprofit['Profit']),
])
# Change the bar mode
fig.update_layout(
    title='Product Profitability',
    xaxis=dict(
        title='Part Profitability',
        tickfont_size=14
    ),
    yaxis=dict(
    title='USD Net Profit'
    ),
    barmode='group'

)



fig.show()








In [17]:

# functionalize^ 
def groupdata(file,group,sumfield):
    df=pd.DataFrame(file)
    productprofit=pd.DataFrame(df.groupby(str(group))[str(sumfield)].sum().reset_index().sort_values(str(sumfield),ascending=False))
    customerprofit

    #ceate visualization

    fig = go.Figure(data=[
        go.Bar(name=str(group) + 'Profitability', x=productprofit[str(group)], y=productprofit[str(sumfield)]),
    ])
    # Change the bar mode
    fig.update_layout(
        title=str(group) + 'Profitability',
        xaxis=dict(
            title= str(group),
            tickfont_size=14
        ),
        yaxis=dict(
        title='USD Net Profit'
        ),
        barmode='group'

    )
    return fig



In [18]:
groupdata(transoutput,'PartName','Profit')

In [19]:
#What do I do about it?


#Based on the reason behind the net profit loss(e.g. lionshare of the OPEX, make a recommendation.)
#come up with a custom set of recommendations

In [20]:
testtable=transoutput

In [21]:
testtable.columns

Index(['TID', 'PartNumber', 'PartName', 'CustomerNumber', 'CustomerName',
       'Revenue', 'COGS', 'Quantity', 'GrossMargin', 'DistributionAC',
       'SellingRelatedAC', 'G&AAC', 'Profit'],
      dtype='object')

In [22]:
fig = go.Figure(data=[go.Table(header=dict(values=list(testtable.columns)),
                 cells=dict(values=round(testtable,2).transpose()))
                     ])
fig.show()

In [24]:
allocation=splitter2('uploads/transaction.csv','uploads/pl.csv')

In [25]:
allocation

,TID,PartNumber,PartName,CustomerNumber,CustomerName,Revenue,COGS,Quantity,GrossMargin,DistributionAC,SellingRelatedAC,G&AAC,Profit
0,100,50,A,50,H,654,478,1,176,83307.86,49984.71,99969.43,-234595.00
1,101,50,A,51,I,474,333,2,141,60379.09,36227.45,72454.91,-170072.45
2,102,50,A,50,H,393,221,3,172,50061.14,30036.69,60073.37,-140990.20
3,103,51,B,50,I,474,412,2,62,60379.09,36227.45,72454.91,-170153.45
4,104,51,B,51,H,393,322,2,71,50061.14,30036.69,60073.37,-141094.20
5,105,51,B,51,I,909,646,2,263,115790.28,69474.17,138948.33,-325976.78
6,106,51,B,51,I,7575,3322,3,4253,964918.99,578951.39,1157902.78,-2712881.16
7,107,51,B,51,I,3737,322,3,3415,476026.70,285616.02,571232.04,-1337145.76
8,108,51,B,51,I,299,131,1,168,38087.23,22852.34,45704.68,-107285.25
9,109,52,C,52,J,473,322,1,151,60251.71,36151.02,72302.05,-169713.78


In [36]:
df=pd.read_csv('/Users/jordanlange/Documents/projects/profitanalysis/uploads1/allocation.csv')


FileNotFoundError: File b'/Users/jordanlange/Documents/projects/profitanalysis/uploads1/allocation.csv' does not exist

In [27]:
df2=pd.DataFrame(df.groupby(str(groupinput))[str(suminput)].sum().reset_index().sort_values(str(suminput),ascending=False))

NameError: name 'groupinput' is not defined

In [28]:
def returnfig(groupinput,suminput):
    df=pd.read_csv('/Users/jordanlange/Documents/projects/profitanalysis/uploads1/allocation.csv')
    df2=pd.DataFrame(df.groupby(str(groupinput))[str(suminput)].sum().reset_index().sort_values(str(suminput),ascending=False))
    fig = go.Figure(data=[go.Bar(name=str(groupinput) + 'Profitability', x=df2[str(groupinput)], y=df2[str(suminput)]),
    ])
    # Change the bar mode
    fig.update_layout(
        title=str(groupinput) + 'Profitability',
        xaxis=dict(
            title= str(groupinput),
            tickfont_size=14
        ),
        yaxis=dict(
        title='USD'+str(suminput)
        ),
        barmode='group'

    )

    # fig2=dcc.graph(
    # id='transaction-graph',
    # figure=fig
    # )  
    return fig

In [31]:
returnfig ('CustomerName','DistributionAC')